In [4]:
library(tidyverse)
library(dimRed)

In [5]:
train = read.csv("train.csv")

In [27]:
## set the seed to make your partition reproducible
set.seed(503)

sub_ind <- sample(seq_len(nrow(train)), size = floor(0.1 * nrow(train)))

sub <- train[sub_ind, ]
rest = train[-sub_ind,]

In [10]:
sub %>% head
sub %>% dim

,title,subject,date,label,s,trump,said,president,t,u,⋯,services,cut,latest,questions,received,rally,peace,oil,access,planned
,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10731,GOP MAJORITY SENATE FINALLY GETS IT RIGHT: Votes To Gut Obamacare And Defund Planned Parenthood,politics,4-Dec-15,Fake,0.09832334,0.0000000,0.09186381,0.08384356,0.0000000,0,⋯,0,0,0.0000000,0,0,0,0,0,0,0
17811,WATCH: Obama PERFECTLY Mocks Trump<U+2019>s Insane Followers That Think He<U+2019>s A Literal Demon,News,12-Oct-16,Fake,0.10403339,0.0000000,0.06074920,0.33267264,0.2492488,0,⋯,0,0,0.0000000,0,0,0,0,0,0,0
20164,Saudi says Iranian talk of rapprochement is laughable,worldnews,5-Sep-17,True,0.16958778,0.0000000,0.27728121,0.00000000,0.0000000,0,⋯,0,0,0.0000000,0,0,0,0,0,0,0
13222,DEMOCRAT DERSHOWITZ Slaps Down Feinstein<U+2019>s Effort to Say Trump Guilty of Obstruction: <U+2018>She doesn<U+2019>t know what she<U+2019>s talking about<U+2019> [Video],politics,4-Dec-17,Fake,0.28167539,0.0000000,0.05482713,0.10008079,0.1499673,0,⋯,0,0,0.0000000,0,0,0,0,0,0,0
18161,Al Gore Has The PERFECT Piece Of Advice For Donald Trump,News,17-Aug-17,Fake,0.36404599,0.2023689,0.08503233,0.00000000,0.1162935,0,⋯,0,0,0.1207856,0,0,0,0,0,0,0
752,Obama Just Threw MAJOR Shade At Trump During Last National Security Speech (VIDEO),News,6-Dec-16,Fake,0.23460812,0.0000000,0.03914199,0.35724660,0.0000000,0,⋯,0,0,0.0000000,0,0,0,0,0,0,0


[1] 3370  604

In [29]:
sub = sub %>% select(-title, - date, -subject)
rest = rest %>% select(-title, - date, -subject)

In [33]:
tune.out <- tune(svm, as.factor(label) ~., data = sub, 
              ranges = list(gamma = 2^(-1:1), cost = 10^(-2:2)),
              tunecontrol = tune.control(sampling = "cross", cross = 5), 
              scale = F
             )

In [34]:
bestmod=tune.out$best.model
summary(bestmod)


Call:
best.tune(method = svm, train.x = as.factor(label) ~ ., data = sub, 
    ranges = list(gamma = 2^(-1:1), cost = 10^(-2:2)), tunecontrol = tune.control(sampling = "cross", 
        cross = 5), scale = F)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  radial 
       cost:  10 

Number of Support Vectors:  1490

 ( 666 824 )


Number of Classes:  2 

Levels: 
 Fake True




In [73]:
errors = function(cm){
    print(paste("Fake Accuracy: ", round(cm[1,1] / (cm[1,1] + cm[2,1]), 4)))
    print(paste("True Accuracy: ", round(cm[2,2] / (cm[1,2] + cm[2,2]),4)))
    print(paste("Overall Accuracy: ", round((sum(diag(cm)) / sum(cm)), 4)))
}

In [35]:
sub_pred= predict(bestmod , sub)

       truth
predict Fake True
   Fake 1733    1
   True    0 1636

In [74]:
cm1 = table(predict=sub_pred, truth=sub$label)
cm1
errors(cm1)

       truth
predict Fake True
   Fake 1733    1
   True    0 1636

[1] "Fake Accuracy:  1"
[1] "True Accuracy:  0.9994"
[1] "Overall Accuracy:  0.9997"


In [36]:
rest_pred = predict(bestmod, newdata = rest)

       truth
predict  Fake  True
   Fake 14791   750
   True  1086 13708

In [75]:
cm2 = table(predict=rest_pred, truth=rest$label)
cm2
errors(cm2)

       truth
predict  Fake  True
   Fake 14791   750
   True  1086 13708

[1] "Fake Accuracy:  0.9316"
[1] "True Accuracy:  0.9481"
[1] "Overall Accuracy:  0.9395"
